<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/database/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers


In [ ]:
import psycopg2
from sentence_transformers import SentenceTransformer
import json

In [ ]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [ ]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

In [ ]:
# Connect to PostgreSQL database
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
conn.commit()

cur.execute("""
DROP SCHEMA IF EXISTS travel_database CASCADE;
CREATE SCHEMA IF NOT EXISTS travel_database;
SET search_path TO travel_database, public;  -- Use public where PostGIS is installed

-- Create a custom type for the Address structure
CREATE TYPE Address AS (
    street TEXT,
    district TEXT,
    city TEXT
);

-- Create the Hotel table with an embedding vector column
CREATE TABLE Hotel (
    hotel_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),  -- PostGIS geography type for location data
    rating DECIMAL(2, 1),
    description TEXT,
    embedding_description vector(768),  -- Using the vector extension for 768-dimensional embeddings
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the Hotel table
CREATE INDEX idx_hotel_address_rating ON Hotel(((address).district), rating);

-- Create the Price table for the Hotel table
CREATE TABLE HotelPrice (
    hprice_id SERIAL PRIMARY KEY,
    hotel_id INT REFERENCES Hotel(hotel_id) ON DELETE CASCADE,
    room_type VARCHAR(255),
    capacity INT,
    price INT NOT NULL
);

-- Create an index on the hotel_id for the HotelPrice table
-- CREATE INDEX hotel_idx_address_district ON Hotel(((address).district));
-- CREATE INDEX hotel_idx_rating ON Hotel(rating);
-- CREATE INDEX hotel_idx_location ON Hotel USING GIST(location);


-- Create the TouristAttraction table with an embedding vector column
CREATE TABLE TouristAttraction (
    attraction_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),  -- PostGIS geography type for location data
    attraction_type VARCHAR(255),
    rating DECIMAL(2, 1),
    tour_duration INTERVAL,
    description TEXT,
    embedding_description vector(768),  -- Using the vector extension for 768-dimensional embeddings
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the TouristAttraction table
-- CREATE INDEX attraction_idx_address_district ON TouristAttraction(((address).district));
-- CREATE INDEX attraction_idx_rating ON TouristAttraction(rating);
-- CREATE INDEX attraction_idx_location ON TouristAttraction USING GIST(location);


-- Create the Price table for the TouristAttraction table
CREATE TABLE AttractionPrice (
    aprice_id SERIAL PRIMARY KEY,
    attraction_id INT REFERENCES TouristAttraction(attraction_id) ON DELETE CASCADE,
    ticket_type VARCHAR(255),
    number_people INT,
    price INT NOT NULL
);

-- Create an index on the attraction_id for the AttractionPrice table
-- CREATE INDEX idx_attractionprice_attractionid ON AttractionPrice(attraction_id);

-- Create the Restaurant table with an embedding vector column
CREATE TABLE Restaurant (
    res_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),  -- PostGIS geography type for location data
    rating DECIMAL(2, 1),
    description TEXT,
    embedding_description vector(768),  -- Using the vector extension for 768-dimensional embeddings
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the Restaurant table
-- CREATE INDEX res_idx_address_district ON Restaurant(((address).district));
-- CREATE INDEX res_idx_rating ON Restaurant(rating);
-- CREATE INDEX res_idx_location ON Restaurant USING GIST(location);
""")
conn.commit()
cur.close()
conn.close()

In [ ]:
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()

# Dữ liệu mẫu
hotels = [
    ("Khách sạn A", "Khách sạn cao cấp gần trung tâm Hà Nội, với các tiện nghi hiện đại và dịch vụ khách hàng xuất sắc.", 4.5),
    ("Khách sạn B", "Khách sạn tiện nghi và thoải mái, lý tưởng cho những ai tìm kiếm sự yên tĩnh và thư giãn sau một ngày dài.", 3.8),
    ("Khách sạn C", "Khách sạn giá rẻ nhưng chất lượng tốt, phù hợp cho các du khách với ngân sách hạn chế.", 4.2),
    ("Khách sạn D", "Khách sạn sang trọng với dịch vụ 5 sao, bao gồm spa, bể bơi và nhà hàng cao cấp.", 4.9),
    ("Khách sạn E", "Khách sạn gần sân bay và thuận tiện di chuyển, cung cấp dịch vụ đưa đón và hỗ trợ 24/7.", 4.1)
]

attractions = [
    ("Tháp Rùa", "Một biểu tượng nổi tiếng của Hà Nội, nằm trên hồ Hoàn Kiếm, rất phổ biến với du khách và người dân địa phương.", "Di tích lịch sử và điểm tham quan nổi bật", 4.7),
    ("Chùa Một Cột", "Chùa cổ kính với kiến trúc độc đáo, nổi tiếng với hình dáng như một bông sen và nằm trên một cột duy nhất.", "Di tích văn hóa và tâm linh", 4.6),
    ("Lăng Bác", "Lăng chủ tịch Hồ Chí Minh, nơi lưu giữ di hài của Hồ Chí Minh, một điểm đến quan trọng và thiêng liêng ở Hà Nội.", "Nơi thờ cúng và lưu giữ di sản", 4.8),
    ("Văn Miếu Quốc Tử Giám", "Trường đại học cổ xưa nhất của Việt Nam, nơi thờ các vị hiền triết và là một biểu tượng của giáo dục truyền thống.", "Di tích giáo dục và văn hóa", 4.9),
    ("Hồ Tây", "Hồ lớn nhất tại Hà Nội, là nơi lý tưởng cho các hoạt động giải trí, đi dạo và thư giãn bên bờ hồ xanh mát.", "Khu vực giải trí và thư giãn", 4.4)
]

restaurants = [
    ("Nhà hàng A", "Nhà hàng phục vụ các món ăn địa phương với hương vị đậm đà, nổi tiếng với món bún chả và phở.", 4.2),
    ("Nhà hàng B", "Nhà hàng phục vụ món Âu và Á, với thực đơn đa dạng và phong cách ẩm thực quốc tế.", 3.9),
    ("Nhà hàng C", "Nhà hàng hải sản tươi ngon, chuyên cung cấp các món hải sản tươi sống và món đặc sản vùng biển.", 4.3),
    ("Nhà hàng D", "Nhà hàng sang trọng với không gian ấm cúng, phù hợp cho các bữa tối lãng mạn và sự kiện đặc biệt.", 4.6),
    ("Nhà hàng E", "Nhà hàng nhanh và tiện lợi, phục vụ các món ăn nhanh và đồ uống, lý tưởng cho bữa trưa gọn gàng.", 4.0)
]


# Kết nối đến cơ sở dữ liệu PostgreSQL
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()
cur.execute("""SET search_path TO travel_database, public;""")

# Chèn dữ liệu vào bảng Hotel
for name, description, rating in hotels:
    embedding = get_embedding(description)
    cur.execute("""
        INSERT INTO Hotel (name, address, location, rating, description, embedding_description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, %s, %s, %s);
    """, (name, rating, description, embedding, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Chèn dữ liệu vào bảng TouristAttraction
for name, description, attraction_type, rating in attractions:
    embedding = get_embedding(description)
    cur.execute("""
        INSERT INTO TouristAttraction (name, address, location, attraction_type, rating, tour_duration, description, embedding_description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, NULL, %s, %s, %s, %s);
    """, (name, attraction_type, rating, description, embedding, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Chèn dữ liệu vào bảng Restaurant
for name, description, rating in restaurants:
    embedding = get_embedding(description)
    cur.execute("""
        INSERT INTO Restaurant (name, address, location, rating, description, embedding_description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, %s, %s, %s);
    """, (name, rating, description, embedding, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Commit các thay đổi
conn.commit()

# Đóng kết nối và con trỏ
cur.close()
conn.close()